In [63]:
import pandas as pd
import numpy as np
import os

In [4]:
# json format
{
    "_id": "MESH1-MESH2",
    "MESH1": "D000001",
    "MESH2": "D002217",
    "UMLS1": "C0000233",
    "UMLS2": "C0000234",
    "combo": ["D000001-D002217", "D00001-UMLS1", "UMLS1-MESH2", "UMLS1-UMLS2"],
    "overal_freq": {
        "total": 23,
        "most_current_10": 10,
        "most_current_5": 5
    },
    "starred_frequency": {
        "total": 5,
        "most_current_10": 2,
        "most_current_5": 1
    },
    "ngd_overall": 0.23, # ngd: normalized google distance
    "ngd_starred": 0.12
}

{'_id': 'MESH1-MESH2',
 'MESH1': 'D000001',
 'MESH2': 'D002217',
 'UMLS1': 'C0000233',
 'UMLS2': 'C0000234',
 'combo': ['D000001-D002217', 'D00001-UMLS1', 'UMLS1-MESH2', 'UMLS1-UMLS2'],
 'overal_freq': {'total': 23, 'most_current_10': 10, 'most_current_5': 5},
 'starred_frequency': {'total': 5, 'most_current_10': 2, 'most_current_5': 1},
 'ngd_overall': 0.23,
 'ngd_starred': 0.12}

In [141]:
def load_json(data_folder):
    
    # returns ngd and ngd*
    def get_ngd(doc):
        N = 2.7e+7 * 20 # from PubMed home page there are 27 million articles; avg 20 MeSH terms per article
        fx = df_freq[df_freq['DUI'] == doc['MESH1']]['Total'].item()
        fy = df_freq[df_freq['DUI'] == doc['MESH2']]['Total'].item()
        fxy = doc['overall_freq']['total']
        fxy_s = doc['starred_freq']['total']

        ngd = (max(np.log(fx),np.log(fy)) - np.log(fxy)) / (np.log(N) - min(np.log(fx),np.log(fy)))
        ngd_s = (max(np.log(fx),np.log(fy)) - np.log(fxy_s)) / (np.log(N) - min(np.log(fx),np.log(fy)))
        return ngd, ngd_s
    
    co_occur = os.path.join(data_folder, 'summary_CoOccurs_2020.txt')
    freq = os.path.join(data_folder, 'MH_freq_counts_2020.txt')
    assert os.path.exists(co_occur)
    assert os.path.exists(freq)
    
    df = pd.read_csv(co_occur, sep='|')
    df_freq = pd.read_csv(freq, sep='|')
    
    timeFrames = ['MBD', 'MED']
    for time in timeFrames:
        loc = 'Freq_' + time
        loc_s = 'StarFreq_' + time
        df[loc] = df[df['TimeFrame(SAB)'] == time]['Freq']
        df[loc_s] = df[df['TimeFrame(SAB)'] == time]['StarFreq(COF)']
    df.fillna(0, inplace=True)
    
    cols = ['Freq', 'StarFreq(COF)', 'Freq_MBD', 'StarFreq_MBD', 'Freq_MED', 'StarFreq_MED']
    df = df.groupby(['DUI1', 'DUI2', 'CUI1', 'CUI2'])[cols].sum().astype(int).reset_index()
    
    for i,row in df.iterrows():
        doc = {'_id' : '-'.join([row[0], row[1]]),
              'MESH1' : row[0], 'MESH2' : row[1], 'UMLS1' : row[2], 'UMLS2' : row[3],
              'combo' : ['-'.join([i,j]) for i in [row[0],row[2]] for j in [row[1],row[3]]],
              'overall_freq' : {'total' : row[4], 'most_current_10' : row[6], 'most_current_5' : row[8]},
              'starred_freq' : {'total' : row[5], 'most_current_10' : row[7], 'most_current_5' : row[9]}
              }
        ngd = get_ngd(doc)
        doc['ngd_overall'] = ngd[0]
        doc['ngd_starred'] = ngd[1]
        yield doc
    

In [76]:
df = pd.read_csv('~/Documents/NIH_CoOccurs/co-occurs/co_occurs_1', sep='|')

In [77]:
df

,DUI1,CUI1,DUI2,CUI2,Freq,StarFreq(COF),Year,TimeFrame(SAB),numNoSH,StarnumNoSH,numSH,BothMain?,Star1OnlyFreq,Star2OnlyFreq,SH1OnlyFreq,SH2OnlyFreq
0,D000001,C0000699,D000067759,C0101725,1,0,2009,MBD,0,0,0,ZN,0,0,1,0
1,D000001,C0000699,D000067856,C1882413,1,0,1990,RST,0,0,0,ZN,0,0,1,0
2,D000001,C0000699,D000067856,C1882413,1,0,1996,RST,0,0,0,ZN,0,1,1,0
3,D000001,C0000699,D000067856,C1882413,1,0,2005,RST,0,0,0,ZN,0,1,1,0
4,D000001,C0000699,D000068299,C0245531,1,0,1995,RST,0,0,0,ZN,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6999994,D000223,C0683269,D046352,C1449593,1,0,2010,MBD,0,0,0,ZN,0,0,0,0
6999995,D000223,C0683269,D046352,C1449593,2,0,2014,MED,1,0,0,ZN,0,0,0,1
6999996,D000223,C0683269,D046352,C1449593,1,0,2015,MED,0,0,0,ZN,1,0,0,0
6999997,D000223,C0683269,D046352,C1449593,1,0,2016,MED,0,0,0,ZN,1,0,1,0


In [79]:
timeFrames = ['MBD', 'MED']

for time in timeFrames:
    loc = 'Freq_' + time
    loc_s = 'StarFreq_' + time
    df[loc] = df[df['TimeFrame(SAB)'] == time]['Freq']
    df[loc_s] = df[df['TimeFrame(SAB)'] == time]['StarFreq(COF)']
df.fillna(0, inplace=True)

In [81]:
cols = ['Freq', 'StarFreq(COF)', 'Freq_MBD', 'StarFreq_MBD', 'Freq_MED', 'StarFreq_MED']

df = df.groupby(['DUI1', 'DUI2', 'CUI1', 'CUI2'])[cols].sum().astype(int).reset_index()
df

,DUI1,DUI2,CUI1,CUI2,Freq,StarFreq(COF),Freq_MBD,StarFreq_MBD,Freq_MED,StarFreq_MED
0,D000001,D000067759,C0000699,C0101725,1,0,1,0,0,0
1,D000001,D000067856,C0000699,C1882413,3,0,0,0,0,0
2,D000001,D000068299,C0000699,C0245531,3,0,0,0,0,0
3,D000001,D000068577,C0000699,C0068475,2,0,0,0,0,0
4,D000001,D000068579,C0000699,C0538927,3,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
1622567,D000223,D046228,C0683269,C1449575,1,0,0,0,1,0
1622568,D000223,D046269,C0683269,C1449584,1,1,0,0,0,0
1622569,D000223,D046288,C0683269,C0848128,1,1,1,1,0,0
1622570,D000223,D046329,C0683269,C0028828,1,0,0,0,0,0


In [127]:
for i,row in df.iterrows():
    doc = {'_id' : '-'.join([row[0], row[1]]),
          'MESH1' : row[0], 'MESH2' : row[1], 'UMLS1' : row[2], 'UMLS2' : row[3],
          'combo' : ['-'.join([i,j]) for i in [row[0],row[2]] for j in [row[1],row[3]]],
          'overall_freq' : {'total' : row[4], 'most_current_10' : row[6], 'most_current_5' : row[8]},
          'starred_freq' : {'total' : row[5], 'most_current_10' : row[7], 'most_current_5' : row[9]}
          }
    ngd = get_ngd(doc)
    doc['ngd_overall'] = ngd[0]
    doc['ngd_starred'] = ngd[1]
    print(doc)
    break

{'_id': 'D000001-D000067759', 'MESH1': 'D000001', 'MESH2': 'D000067759', 'UMLS1': 'C0000699', 'UMLS2': 'C0101725', 'combo': ['D000001-D000067759', 'D000001-C0101725', 'C0000699-D000067759', 'C0000699-C0101725'], 'overall_freq': {'total': 1, 'most_current_10': 1, 'most_current_5': 0}, 'starred_freq': {'total': 0, 'most_current_10': 0, 'most_current_5': 0}, 'ngd_overall': 0.785288051205346, 'ngd_starred': inf}


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: RuntimeWarning: divide by zero encountered in log
  # Remove the CWD from sys.path while we load stuff.


In [125]:
df_freq = pd.read_csv('~/Documents/NIH_CoOccurs/MH_freq_counts_2020.txt', sep='|')
df_freq[df_freq['DUI'] == doc['MESH2']]['Total'].item()

3518

In [138]:
# returns ngd and ngd*
def get_ngd(doc):
    N = 2.7e+7 * 20 # from PubMed home page there are 27 million articles; avg 20 MeSH terms per article
    fx = df_freq[df_freq['DUI'] == doc['MESH1']]['Total'].item()
    fy = df_freq[df_freq['DUI'] == doc['MESH2']]['Total'].item()
    fxy = doc['overall_freq']['total']
    fxy_s = doc['starred_freq']['total']
    
    print(fx, fy, fxy, fxy_s)
    
    ngd = (max(np.log(fx),np.log(fy)) - np.log(fxy)) / (np.log(N) - min(np.log(fx),np.log(fy)))
    ngd_s = (max(np.log(fx),np.log(fy)) - np.log(fxy_s)) / (np.log(N) - min(np.log(fx),np.log(fy)))
    
    return ngd, ngd_s

In [139]:
get_ngd(doc)

11819 3518 1 0


0.785288051205346

In [140]:
np.log(0)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log
  """Entry point for launching an IPython kernel.


-inf